# Commom Data Types

## Determining data types from existing tables

~~~~sql
 -- Select all columns from the TABLES system database
 SELECT * 
 FROM INFORMATION_SCHEMA.TABLES
 -- Filter by schema
 WHERE table_schema = 'public';
 
  -- Select all columns from the COLUMNS system database
 SELECT * 
 FROM INFORMATION_SCHEMA.COLUMNS 
 WHERE table_name = 'actor';
~~~~

~~~~sql
SELECT column_name,
       data_type
FROM INFORMATION_SCHEMA.COLUMNS
WHERE column_name in ('title', 'description', 'special_features')
AND table_name = 'film';
~~~~

## Date and Time Data Types

### TIMESTAMP data types
TIMESTAMPs contain both a date value and a time value with microsecond precision.
- These data types are very common because they can be used to record an exact point in time like when a payment was made or a record was last updated.

### DATE and TIME types
- DATE and TIME types are essentially the date and time values of the TIMESTAMP. 
- DATE types contain a date value with no time of day
- TIME types contain the time of day but without a date.

### INTERVAL data types
- INTERVAL types store date and time data as a period of time in years, months, days, hours, seconds, etc.
- INTERVALs are useful when you want to do arithmetic on date and time columns. 

~~~~sql
SELECT
 	-- Select the rental and return dates
	rental_date,
	return_date,
 	-- Calculate the expected_return_date
	rental_date + INTERVAL '3 days' AS expected_return_date
FROM rental;
~~~~

## Arrays
To create an ARRAY type, you simply need to add "square brackets" to the end of the data type that you want to make an array.

For instance, the following table has an email column which will be a nested array of text data to store the email type and the address for a given student_id. The test_scores column will contain an array of integer values representing the numeric test score.
~~~~sql
CREATE TABLE grades (
    student_id int,
    email text[][]
    test_scores int[]
    );
~~~~

To INSERT with ARRAYS:
~~~~sql
INSERT INTO grades
VALUES (1,
        '{{"work","work1@datacamp.com"}, {"other", "other1@datacamp.com"}}',
        '{92, 85, 96, 88}' );
~~~~

Accessing ARRAYs
- Accessing arrays in PostgreSQL is very similar to accessing arrays in other programming languages. For email, you can get the first element of the first array by using the array notation you see here with index values of 1. Note that PostgreSQL array indexes start with one and not zero.
~~~~sql
SELECT email[1][1] AS type,
       email[1][2] AS address,
       test_scores[1],
FROM grades;
~~~~

The ANY function allows you to search for a value in any index position of an ARRAY. Here's an example.
~~~~sql
SELECT
  title, 
  special_features 
FROM film 
-- Modify the query to use the ANY function 
WHERE 'Trailers' = ANY (special_features);
~~~~

The contains operator @> operator is alternative syntax to the ANY function and matches data in an ARRAY using the following syntax.
~~~~sql
SELECT 
  title, 
  special_features 
FROM film 
-- Filter where special_features contains 'Deleted Scenes'
WHERE special_features @> ARRAY['Deleted Scenes'];
~~~~

# Working with DATE/TIME Functions and Operators

## Overview of basic arithmetic operators

### Adding and subtracting date / time data

- Subtract date values, the result that is returned is an integer data type.
- When adding integers to date values, the implied precision is days.
- When we perform the same operation on timestamp data types as you see in this example, we get an INTERVAL as the result.
- The AGE function allows us to calculate the difference between two timestamps. The AGE function takes two timestamp arguments and subtracts the first argument from the second and returns an INTERVAL as a result. 

~~~~sql
SELECT f.title, f.rental_duration,
    -- Calculate the number of days rented
	AGE(r.return_date, r.rental_date) AS days_rented
FROM film AS f
	INNER JOIN inventory AS i ON f.film_id = i.film_id
	INNER JOIN rental AS r ON i.inventory_id = r.inventory_id
ORDER BY f.title;
~~~~

### Current thimestap

~~~~sql
SELECT NOW(); -- NOW() allows you to retrieve a timestamp value for the current date and time at the microsecond precision with time zone
SELECT NOW()::timestamp; -- timestamp whithout the time zone (PostgreSQL standart)
SELECT CAST(NOW() as timestamp); -- timestamp whithout the time zone
~~~~

~~~~sql
SELECT CURRENT_DATE;
SELECT CURRENT_TIME;
~~~~

### Extracting and transforming date/time data
- EXTRACT(field FROM source)
- DATE_PART('field', source)
- DATE_TRUNC() function will truncate timestamp or interval data type return a timestamp or interval at a specified precision. 
    ~~~~sql
    SELECT DATE_TRUNC('year', rental_date) AS rental_year
    ~~~~

# Reformatting string and character data

## The string concatenation operator
String concatenation allows you to merge two or more strings together to form a single combined string. In the following example, you see how we can combine two separate columns from the customer table, first_name and last_name, to create a new column called full_name.

~~~~sql
SELECT first_name,
       last_name,
       first_name || ' ' || last_name AS full_name
FROM customer
~~~~
PostgreSQL also has a built-in function for string concatenation.The CONCAT() function accepts one or more parameters and returns the concatenated string as the result. Each parameter can be a column from a database or a literal value separate by a comma. 

~~~~~sql
SELECT CONCAT(first_name, ' ', last_name) AS full_name
FROM customer;
~~~~~

## Reformarting
~~~~sql
SELECT
    UPPER(email) -- to uppercase the string
FROM customer;

SELECT
    LOWER(email) -- to lowercase the string
FROM customer;

SELECT
    INITCAP(title) -- convert a string to title case
FROM customer;
~~~~

## Replacing characters

~~~~sql
SELECT
    REPLACE(description, 'A Astounding', 'An Astouding') as description
FROM film;
~~~~

## Reversing
The REVERSE function does just what you think it does...it accepts a string as its only parameter and returns the same string in reverse order as you see when we use the function to reverse the title column of the film table.

~~~~sql
SELECT  title,
        REVERSE(title)
FROM  film AS f;
~~~~

## Determining the length of a string
~~~~sql
SELECT title,
       CHAR_LENGTH(title), -- LENGTH(title) is a synonymous function
FROM film;
~~~~

## Finding the position of a character in a string
The POSITION or STRPOS function returns an integer which represents the number of characters from left to right before the search string is located. 
~~~~sql
SELECT 
     email,
     STRPOS(email, '@')
FROM customer;
~~~~

The LEFT function allows you to extract the first 'n' characters of a string.
~~~~sql
SELECT
    LEFT(description, 50) -- extracts the first 50 characters
FROM film;
The RIGHT function is very similar to LEFT but as you might expect it extracts the last "n" characters of a string.
SELECT
    RIGHT(description, 50) -- extracts the last 50 characters
FROM film;
~~~~

SUBSTRINGs or SUBSTR 
~~~~sql
SELECT substring(description, 10, 50) -- selects a substring from the case position 10 with a length of 50
FROM film;
~~~~

Subsstring can be combined:
~~~~sql
SELECT
    SUBSTRING(email FROM 0 FOR POSITION('@' IN email))
FROM customer;
~~~~

~~~~sql

SELECT
  -- Extract the characters to the left of the '@'
  LEFT(email, POSITION('@' IN email)-1) AS username,
  -- Extract the characters to the right of the '@'
  SUBSTRING(email FROM POSITION('@' IN email)+1 FOR LENGTH(email)) AS domain
FROM customer;
~~~~

## Truncating and padding string data
The TRIM function will remove characters from either the start or end of the string or both and accepts three parameters. 
- The first parameter is optional and specifies whether you want to remove characters from the beginning or end of a string or both. 
    - If this parameter is omitted, the default value is both. 
- The second parameter which is also optional specifies the characters to be removed from the string.
    - If this parameter is omitted, the default value is a blank space.
- The third parameter is the string that you wish to trim.

The LTRIM and RTRIM functions are analogous to TRIM but only remove characters from either the beginning OR the end of the string, not both.

The function LPAD appends a character or string to another string by a specified number of characters. 
- If you omit the third parameter in the LPAD function the string will be padded with a space character by default. 
- When the length parameter is less than the original length of the string the result returned will be truncated.

The RPAD function is analogous to LPAD but will pad the string with characters to the right.

~~~~sql
-- Concatenate the padded first_name and last_name 
SELECT 
	RPAD(first_name, LENGTH(first_name)+1) || last_name AS full_name
FROM customer;
~~~~

# Full-text Search 

## LIKE operator
The LIKE operator can be used in a WHERE clause to search for a pattern in a column.

- '%' is a substitute for a **sequence** of characters
- '-' helps you match a **single** character

In this case we used to obtain all employees that the name begins with 'Mar'.
~~~~sql
SELECT *
FROM employees
WHERE first_name LIKE('Mar%');
~~~~

## Full-text search
Full text search can get complex but even a basic full text search query can be a very powerful tool.
- Full text search provides a means for performing natural language queries of text data by using stemming, fuzzy string matching to handle spelling mistakes and a mechanism to rank results by similarity to the search string.


~~~~sql
SELECT title
FROM film
WHERE to_tsvector(title) @@ to_tsquery('elf');
~~~~

- The WHERE clause of the query uses the match operator to compare the values returned by two built-in functions to perform the search, to_tsvector and to_tsquery. These functions convert text and string data to a tsvector data type which is a sorted list of words that have been normalized into variants of the same word. These variants are called `lexemes`.

# User-Definde Data Types

A user-defined data type is created using the CREATE TYPE command which registers the type in a system table and makes it available to be used anywhere PostgreSQL expects a type name.
Enumerated data types or enums allow you to define a custom list of values that are never going to change, like the days of the week. As you can see in this example, a new data type called dayofweek is defined as an ENUM using the CREATE TYPE command with a comma separated list of the days of the week.

~~~~sql
CREATE TYPE dayofweek AS ENUM (
    'Monday',
    'Tuesday',
    'Wednesday',
    'Thrusday',
    'Friday',
    'Saturday',
    'Sunday'
    );
~~~~

Getting information about user-defined data types
~~~~sql
SELECT typname, typcategory
FROM pg_type
WHERE typname='dayofweek';
~~~~

# User-Define Functions
Another way to extend the capabilities of your PostgreSQL database is with user-defined functions. A user-defined function is the PostgeSQL equivalent of a stored procedure where you can bundle several SQL queries and statements together into a single package using the CREATE FUNCTION command. 

# Extensions

Post PostgreSQL distributions come bundled with a common set of widely used and supported extensions from the community that can be used by simply enabling them. Here are a few common extensions:
- PostGIS adds support for allowing location queries to be run in SQL. 
- PostPic allows for image processing within the database. fuzzystrmatch and pg_trgm provide functions that extend full text search capabilities by finding similarities between strings.

Available Extensions
~~~~sql
SELECT name
FROM pg_available_extensions;
~~~~
Installed Extensions
~~~~sql
SELECT extname
FROM pg_extensions;
~~~~

~~~~sql
--Enable the fuzzystrmatch extension
CREATE EXTENSION IF NOT EXISTS fuzzystrmatch;
--Confirm that fuzzstrmatch has been enabledS
ELECT extname FROM pg_extension;
~~~~